In [4]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [10]:
import pandas as pd

# Create the distance matrix as a list of lists
distance_matrix = [
    [0, 318, 164, 374, 463, 387, 504, 652, 568],
    [318, 0, 210, 456, 518, 650, 362, 677, 345],
    [164, 210, 0, 251, 382, 415, 415, 518, 534],
    [374, 456, 251, 0, 147, 208, 269, 322, 752],
    [463, 518, 382, 147, 0, 270, 248, 215, 800],
    [387, 650, 415, 208, 270, 0, 456, 346, 869],
    [504, 362, 415, 269, 248, 456, 0, 355, 684],
    [652, 677, 518, 322, 215, 346, 355, 0, 984],
    [568, 345, 534, 752, 800, 869, 684, 984, 0]
]

# List of cities
cities = [
    "Baton Rouge", "Little Rock", "Jackson", "Montgomery", "Atlanta",
    "Tallahassee", "Nashville", "Columbia", "Oklahoma City"
]

distance = pd.DataFrame(distance_matrix, index=cities, columns=cities)

demands = {
    "Baton Rouge": 4590,
    "Little Rock": 3055,
    "Jackson": 6205,
    "Montgomery": 5080,
    "Atlanta": 10912,
    "Tallahassee": 22244,
    "Nashville": 7051,
    "Columbia": 5282,
    "Oklahoma City": 8023
}
p = 4

In [11]:
m = gp.Model("p-median")
y = m.addVars(cities, cities, vtype=GRB.BINARY, name="Assignment")
x = m.addVars(cities, vtype=GRB.BINARY, name="Facility Location")
m.update()

In [12]:
all_cities_assigned = m.addConstrs((gp.quicksum(y[i, j] for j in cities) == 1 for i in cities), name="All Assigned")
only_designated_facilities = m.addConstrs((y[i, j] <= x[j] for i in cities for j in cities), name="Assign to Facility")
p_constraint = m.addConstr(gp.quicksum(x[j] for j in cities) == p, name="p_constraint")
m.update()

In [13]:
m.setObjective(gp.quicksum(demands[i] * distance.loc[i, j] * y[i, j] for i in cities for j in cities), GRB.MINIMIZE)
m.update()

In [14]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 91 rows, 90 columns and 252 nonzeros
Model fingerprint: 0x0999ea9f
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [6e+05, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 3.140071e+07
Presolve time: 0.02s
Presolved: 91 rows, 90 columns, 252 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 5.025348e+06, 27 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    5025348.0000